In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Biosppy for signal processing
from biosppy.signals import ecg

# Constants
SAMPLING_RATE = 500  # Hz
ecg_folder = "../../../../Datasets/12-lead electrocardiogram database/ECGData"
diagnostics_file = "../../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Label mapping
rhythm_mapping = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}


# Feature extraction function
def extract_comprehensive_features(ecg_signal):
    """
    Extract comprehensive features from entire ECG signal
    """
    try:
        ecg_processed = ecg.ecg(signal=ecg_signal, sampling_rate=SAMPLING_RATE, show=False)
        rpeaks = ecg_processed['rpeaks']
        heart_rate = ecg_processed['heart_rate']

        features = [
            np.mean(ecg_signal),
            np.std(ecg_signal),
            np.median(ecg_signal),
            np.min(ecg_signal),
            np.max(ecg_signal),
            len(rpeaks),
            np.mean(heart_rate) if len(heart_rate) > 0 else 0,
            np.std(heart_rate) if len(heart_rate) > 0 else 0,
            np.mean(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,
            np.std(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,
            np.percentile(ecg_signal, 25),
            np.percentile(ecg_signal, 75),
        ]

        return features

    except Exception as e:
        print(f"Error processing signal: {e}")
        return [0] * 12


# Dataset preparation function
def prepare_dataset(ecg_folder, diagnostics_df):
    signals, signal_features, signal_labels = [], [], []

    for _, row in diagnostics_df.iterrows():
        file_name = row['FileName']
        rhythm_label = row['Rhythm']

        if pd.isnull(rhythm_label) or rhythm_label not in rhythm_mapping.values():
            continue

        ecg_file = os.path.join(ecg_folder, f"{file_name}.csv")
        if not os.path.exists(ecg_file):
            continue

        ecg_data = pd.read_csv(ecg_file, header=0).iloc[:, 1].values
        ecg_data = ecg_data.astype(float)

        features = extract_comprehensive_features(ecg_data)
        signals.append(ecg_data)
        signal_features.append(features)
        signal_labels.append(rhythm_label)

    return np.array(signals), np.array(signal_features), np.array(signal_labels)


# Load and process data
diagnostics_df = pd.read_excel(diagnostics_file)
diagnostics_df['Rhythm'] = diagnostics_df['Rhythm'].map(rhythm_mapping)

raw_signals, signal_features, signal_labels = prepare_dataset(ecg_folder, diagnostics_df)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(signal_labels)
onehot_labels = to_categorical(encoded_labels)

feature_scaler = StandardScaler()
scaled_features = feature_scaler.fit_transform(signal_features)


# Create MLP model
def create_mlp_model(input_shape, num_classes):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model


# Stratified K-Fold Cross-Validation with optimizer tuning
optimizers = ['adam', 'sgd', 'rmsprop', 'adagrad']
results = {}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for optimizer in optimizers:
    print(f"\nTesting optimizer: {optimizer}")
    cv_scores = []

    for train_index, test_index in skf.split(scaled_features, encoded_labels):
        X_train, X_test = scaled_features[train_index], scaled_features[test_index]
        y_train, y_test = onehot_labels[train_index], onehot_labels[test_index]

        model = create_mlp_model(input_shape=X_train.shape[1], num_classes=onehot_labels.shape[1])
        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, verbose=1)
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
        cv_scores.append(accuracy)

        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(y_test, axis=1)

        print("\nClassification Report:")
        print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))

    results[optimizer] = np.mean(cv_scores)
    print(f"Optimizer: {optimizer} - Mean CV Accuracy: {np.mean(cv_scores)}")

print("\nFinal Results:")
for opt, acc in results.items():
    print(f"{opt}: {acc}")

2024-11-29 05:55:14.292203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 05:55:14.303053: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 05:55:14.306343: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 05:55:14.314732: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-29 05:55:14.832415: W tensorflow/compiler/tf2


Testing optimizer: adam


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1732838411.739296  140405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732838411.771635  140405 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732838411.

Epoch 1/100


I0000 00:00:1732838412.532151  143409 service.cc:146] XLA service 0x7addbc00a190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732838412.532179  143409 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-11-29 06:00:12.561107: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-29 06:00:12.671466: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


...

107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2084 - loss: 1.3888 - val_accuracy: 0.3832 - val_loss: 1.3147
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step - accuracy: 0.3771 - loss: 1.3335 - val_accuracy: 0.4624 - val_loss: 1.2939
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.5260 - loss: 1.2817 - val_accuracy: 0.4789 - val_loss: 1.2747
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - accuracy: 0.5830 - loss: 1.2387 - val_accuracy: 0.4930 - val_loss: 1.2550
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step - accuracy: 0.6024 - loss: 1.2012 - val_accuracy: 0.4971 - val_loss: 1.2337
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step - accuracy: 0.6259 - loss: 1.1491 - val_accuracy: 0.5188 - val_loss: 1.2071
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.6395 - loss: 1.1089 - val_accuracy: 0.5399 - val_loss: 1.1804
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step - accuracy: 0.6390 - loss: 1.0702 - val_a

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2258 - loss: 1.5292 - val_accuracy: 0.2758 - val_loss: 1.3381
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step - accuracy: 0.3356 - loss: 1.3544 - val_accuracy: 0.4883 - val_loss: 1.2563
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - accuracy: 0.5067 - loss: 1.2526 - val_accuracy: 0.5264 - val_loss: 1.1878
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step - accuracy: 0.5781 - loss: 1.1582 - val_accuracy: 0.5939 - val_loss: 1.1252
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.6234 - loss: 1.0690 - val_accuracy: 0.6731 - val_loss: 1.0675
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - accuracy: 0.6479 - loss: 1.0009 - val_accuracy: 0.6942 - val_loss: 1.0126
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step - accuracy: 0.6555 - loss: 0.9495 - val_accuracy: 0.7013 - val_loss: 0.9651
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.6816 - loss: 0.8950 - val_a

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.3286 - loss: 1.3407 - val_accuracy: 0.3838 - val_loss: 1.3172
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step - accuracy: 0.5312 - loss: 1.2417 - val_accuracy: 0.3908 - val_loss: 1.2603
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.5529 - loss: 1.1595 - val_accuracy: 0.3950 - val_loss: 1.2142
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 723us/step - accuracy: 0.5897 - loss: 1.0729 - val_accuracy: 0.3985 - val_loss: 1.1775
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - accuracy: 0.5850 - loss: 1.0108 - val_accuracy: 0.4073 - val_loss: 1.1478
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step - accuracy: 0.5828 - loss: 0.9654 - val_accuracy: 0.4120 - val_loss: 1.1235
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - accuracy: 0.5998 - loss: 0.9310 - val_accuracy: 0.4196 - val_loss: 1.1031
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - accuracy: 0.6153 - loss: 0.8850 - val_a